# 주유소 10곳이상 기름값 지도위에 표시하기

**오피넷 - 경로별**   
- 출발지: 서울, 연신내역
- 도착지: 서울, 호서직업전문학교 - 주유소 3개
- 도착지: 서울, 서울역 - 주유소 1개
- 도착지: 서울, 김포국제공항 - 주유소 4개
- 도착지: 서울, 여의도역 - 주유소 5개

In [1]:
import sys
import time
import numpy as np
import pandas as pd

# Unix style pathname pattern expansion
from glob import glob

# A browser automation framework and ecosystem.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller 

# Beautiful Soup is a library that makes it easy to scrape information from web pages.
from bs4 import BeautifulSoup

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm
from tqdm import tqdm_notebook

## Crawling

In [2]:
path = chromedriver_autoinstaller.install() 
driver = webdriver.Chrome(path)
driver.switch_to.window(driver.window_handles[0])

# 메인페이지 접속
driver.get("http://www.opinet.co.kr/user/main/mainView.do")

# 싼주유소찾기 이동
wait = WebDriverWait(driver, 10)
quick_ul = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#quick_ul')))
quick_ul.find_element_by_css_selector('li:nth-child(2) a').click()

# 경로별 페이지 이동
wait = WebDriverWait(driver, 10)
header_path_gsi = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#container .header_path_gsi')))
header_path_gsi.find_element_by_css_selector('li.mid2 a').click()

# 출발지, 도착지 설정
destinations = ['호서직업전문학교', '서울역', '김포국제공항', '여의도역']
departures = ['연신내'] * len(destinations)

for index, destination in tqdm(enumerate(destinations)):

    print('')
    print(destination)

    # 경로별 페이지 설정
    wait = WebDriverWait(driver, 10)
    form = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#searrouteVO')))

    # 경로별 설정 - 출발지
    departure = departures[index]

    element = form.find_element_by_id('search1_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD1')).select_by_visible_text('서울')
    form.find_element_by_id('poiName1').send_keys('')
    form.find_element_by_id('poiName1').send_keys(departure)
    form.find_element_by_id('poiName1').send_keys(Keys.ENTER)

    # 경로별 설정 - 도착지
    element = form.find_element_by_id('search2_2')
    driver.execute_script("arguments[0].style.display = 'none';", element)

    time.sleep(1)
    Select(form.find_element_by_id('SIDO_CD2')).select_by_visible_text('서울')
    form.find_element_by_id('poiName2').send_keys('')
    form.find_element_by_id('poiName2').send_keys(destination)
    form.find_element_by_id('poiName2').send_keys(Keys.ENTER)

    # 경로별 설정 - 조회
    time.sleep(3)
    driver.find_element_by_id('searrouteSelect').click()

    # 경로별 설정 - 데이터 
    time.sleep(3)
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.ID, 'osList')))
    raw = table.find_elements_by_css_selector('tr')
    
    data = []

    for row in raw:

        time.sleep(1)
        row.find_element_by_css_selector('a').click()
        
        time.sleep(3)
        os_dtail_info = driver.find_element_by_id('os_dtail_info')

        tmp = {
            '이름': { 'id':'os_nm', 'type':'string', 'value':'' },
            '전화번호': { 'id':'phn_no', 'type':'string', 'value':'' },
            '주소': { 'id':'rd_addr', 'type':'string', 'value':'' },
            '상표': { 'id':'poll_div_nm', 'type':'string', 'value':'' },
            '고급휘발유': { 'id':'bo34_p', 'type':'integer', 'value':'' },
            '보통휘발유': { 'id':'b027_p', 'type':'integer', 'value':'' },
            '경유': { 'id':'d047_p', 'type':'integer', 'value':'' },
            '실내등유': { 'id':'c004_p', 'type':'integer', 'value':'' },
            '자동차부탄': { 'id':'k015_p', 'type':'integer', 'value':'' },
        }

        time.sleep(1)
        for k, values in tmp.items():
            try:
                tmp[k].update({ 'value': os_dtail_info.find_element_by_id(values['id']).text })
            except Exception as e:
                if tmp[k]['type'] == 'integer':
                    tmp[k].update({ 'value': np.nan })
                print(k, e, sep=', ', end='\n')
        
        tmp2data = {}
        for k, values in tmp.items():
            tmp2data.update({ k: values['value'] })

        data.append(tmp2data)

    # 경로별 설정 - 저장
    df = pd.DataFrame(data)
    df.to_csv('./data/fuel-{}.csv'.format(index), sep=',', encoding='utf8', index=False)

    # to refresh the browser
    time.sleep(3)
    driver.refresh()

driver.close() 
driver.quit()

0it [00:00, ?it/s]
호서직업전문학교
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

1it [00:27, 27.96s/it]
서울역
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

2it [00:45, 24.80s/it]
김포국제공항
고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="bo34_p"]"}
  (Session info: chrome=87.0.4280.66)

고급휘발유, Me

## Merging DataFrame

In [3]:
pd.read_csv('./data/fuel-0.csv')

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN


In [6]:
tmp_raw = []

for filepath in glob('./data/*.csv'):
    tmp = pd.read_csv(filepath)
    tmp_raw.append(tmp)

df = pd.concat(tmp_raw)
df.index = np.arange(0, len(df))
df.to_csv('./data/opinet.csv', sep=',', encoding='utf8', index=False)
df

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN
3,(주)창원이엔지 홍제지점,02-379-0753,서울 서대문구 통일로 455 (홍제동),현대오일뱅크,NaN,"1,299","1,099",NaN,NaN
4,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN
5,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN
6,(주)한진 김포공항주유소,02-2662-0422,서울 강서구 하늘길 259,S-OIL,NaN,"1,319","1,128",NaN,NaN
7,세원제1주유소,02-3663-5151,서울 강서구 양천로 311 (가양동),SK에너지,NaN,"1,389","1,189",NaN,NaN
8,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN
9,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN
